In [2]:
import itertools
import copy
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader, random_split
import torch.nn as nn
import pickle
import random
from sklearn.model_selection import train_test_split
import time
from joblib import Parallel, delayed
import pandas as pd
import sys
import os
import seaborn as sns 
current = os.path.dirname(os.path.realpath('plotting.py'))
parent = os.path.dirname(current)
sys.path.append(parent)
import functions.plotting as NNplt
from functions.rnn_cryptic import generate_sequences, convert_seq2inputs,\
                                  pad_seqs_2step, pad_seqs_1step, calculate_output, onehot2seq 

from sklearn.metrics import r2_score

In [3]:
class OneStepRNN(nn.Module):

    def __init__(self, input_size, output_size, hidden_size, num_layers):
        super(OneStepRNN, self).__init__()
        # Define parameters
        self.rnn = torch.nn.RNN(input_size=input_size,
                        hidden_size=hidden_size,
                        num_layers= num_layers,
                        batch_first=True)
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        # Define the layers
        self.input2hidden = nn.Linear(input_size + self.hidden_size, self.hidden_size)
        self.fc1tooutput = nn.Linear(self.hidden_size, output_size)

    def forward(self, x, hidden):
        combined = torch.cat((x, hidden), dim=0) ## dim = 1??
        self.hidden = nn.functional.relu(self.input2hidden(combined))
        self.output = self.fc1tooutput(self.hidden)
        #return self.output.view(-1,output_size), self.hidden
        return self.output, self.hidden

    def get_activations(self, x, hidden):
        self.forward(x, hidden)  # update the activations with the particular input
        return self.hidden, self.output #, self.fc1_activations

    def get_noise(self):
        return self.hidden_noise

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)[0]


def train(sequence,label,model,optimizer,criterion):
    model.train()
    optimizer.zero_grad()
    #Read each cue in and keep hidden state for next cue
    hidden = model.initHidden()
    batch_out = []
    for batchseq in sequence:
        for i in range(len(batchseq)):
            output, hidden = model.forward(batchseq[i], hidden)
        batch_out.append(output)
        #Compare final output to target
    batch_out = torch.cat(batch_out)
    loss = criterion(batch_out,label)#.long())

    #Back-propagate
    loss.backward()
    optimizer.step()

    return batch_out, loss.item()

def run_acc(model,optimizer,criterion, train_data, test_data, epochs, verbose = False):
    
    loss_history = np.empty((0,1))
    all_accs = np.empty((0,len(test_data)))
    for epoch in range(epochs):
        lossTotal = 0
        for i, (x,y) in enumerate(train_data):
            output, loss = train(x,y,model,optimizer,criterion)
            lossTotal += loss # add MSE -> sum of square errors 
        loss_history = np.vstack([loss_history, lossTotal])
        acc = test_acc(model, test_data, hidden_size)
        all_accs = np.vstack([all_accs,acc])

    print(f'loss: {round(lossTotal,1)} ')
    return loss_history, all_accs

def test_acc(model, testdata, hidden_size, verbose = False):
    model.eval()
    accs = np.empty((1, 0))
    for testset in testdata:
        batch_correct = []
        for x,y in testset:
            correct = 0
            for i in range(len(x)):
                hidden = torch.zeros(1, hidden_size)[0]
                for step in x[i]:
                    hidden, y_hat = model.get_activations(step,hidden)
                correct += sum(torch.round(y[i]) == torch.round(y_hat)).item()
            batch_correct.append(correct/len(y))
        acc = np.mean(batch_correct)
        accs = np.append(accs, [acc])
    if verbose:
        print('test accuracy: %f ' % (acc))
    return accs

def test_preds(model, testdata, hidden_size, suffix = ''):
    model.eval()
    preds = []
    labs = []
    trials = []
    for testset in testdata:
        batch_correct = []
        for x,y in testset:
            correct = 0
            for i in range(len(x)):
                hidden = torch.zeros(1, hidden_size)[0]
                for step in x[i]:
                    hidden, y_hat = model.get_activations(step,hidden)
                preds.append(y_hat.detach().item())
                labs.append(y[i].detach().item())
            trials.append(str(onehot2seq(x)))
    df = pd.DataFrame({'trial':trials, 'label'+suffix:labs, 'pred'+suffix: preds})
    return df 


def shuffle_weights(model):
    model2 = OneStepRNN(input_size, output_size, hidden_size, num_layers)
    mod_dict = model.state_dict()
    shuffled_dict = {layer: shuffle_tensor(val) for layer, val in mod_dict.items()}
    model2.load_state_dict(shuffled_dict)
    return model2

def shuffle_tensor(t):
    idx = torch.randperm(t.nelement())
    t = t.view(-1)[idx].view(t.size())
    return t

def run_sim(train_trials, test_trials):
    model = OneStepRNN(input_size, output_size, hidden_size, num_layers)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learningRate)
    loss1, acc1 = run_acc(model,optimizer,criterion, train_trials, test_trials, epochs)
    return loss1, acc1, model

def run_sims(i, train_trials, test_trials):
    print('########## rep', i, '#########')
    model = OneStepRNN(input_size, output_size, hidden_size, num_layers)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learningRate)
    loss1, acc1 = run_acc(model,optimizer,criterion, train_trials[0], test_trials, epochs)
    loss2, acc2 = run_acc(model,optimizer,criterion, train_trials[1], test_trials, epochs)
    losses = np.vstack([loss1,loss2])
    accs = np.vstack([acc1,acc2])
    return losses, accs, model


def seq_acc(model, testdata, hidden_size, rep = 10, verbose = False):

    model.eval()
    trials = []
    accs = []
    for j, (x,y) in enumerate(testdata):
        for i in range(len(x)):
            hidden = torch.zeros(1, hidden_size)[0]
            for step in x[i]:
                hidden, y_hat = model.get_activations(step,hidden)
            correct = sum(torch.round(y[i]) == torch.round(y_hat)).item() 
            accs.append(float(correct))
            curr_trial = onehot2seq(x)
            trials.append(curr_trial)
    df = pd.DataFrame({'trial':trials, 'acc':accs})
    df['trial'] = df['trial'].apply(str)
           
    return df.sort_values(by = 'acc',ascending = False)

def change_padder(seqs, cue_dict, xval):
    """ changes value of X from zero to a new value
        and recalculates sequence output"""
    cue_dict['X'] = xval
    inps = [s[:-1] for s in seqs]
    for inp in inps:
        inp.append(calculate_output(inp, cue_dict, bidmas = False))

    return inps, cue_dict


# ----------
# plotting
# ----------

def plotNNs(loss_arrays, acc_arrays, labels, colors, title, shift = 0):

    fig, axs = plt.subplots(2, 1)
    loss_cols = ['blue', 'darkblue']
    loss_labs = ['loss_with_primitive', 'loss_without_primitive']
    for i, arr in enumerate(loss_arrays):
        x = np.arange(0,arr.shape[0],1) + shift
        mn = arr.mean(axis=1)
        errs = arr.std(axis=1)
        
        axs[0].plot(x, mn, label = loss_labs[i], color = loss_cols[i])
        axs[0].fill_between(x, mn - errs, mn + errs, alpha = 0.3, facecolor = loss_cols[i])
    
    axs[0].set_xlabel('epoch')
    axs[0].set_ylabel('loss')
    axs[0].legend()
    
    for i, arr in enumerate(acc_arrays):
        x = np.arange(0,arr.shape[0],1) + shift
        mn = arr.mean(axis=1)
        errs = arr.std(axis=1)
        
        axs[1].plot(x, mn, label = labels[i], color = colors[i])
        axs[1].fill_between(x, mn - errs, mn + errs, alpha = 0.3, facecolor = colors[i])
    
    axs[1].set_xlabel('epoch')
    axs[1].set_ylabel('accuracy')
    axs[1].legend()

    fig.suptitle(title, fontsize=10)

In [4]:
# params

num_classes = 22
batchsize=1
# specs
input_size = num_classes
output_size = 1
num_layers = 1
hidden_size = 20


In [5]:
num_inputs = 7

total_syms = ['A','B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P']
all_syms = total_syms[:num_inputs]
all_input_vals = list(np.arange(2,18))
input_vals = random.sample(all_input_vals,num_inputs)
# randomly select values for each input
cue_dict = {}
for i, s in enumerate(all_syms):
    cue_dict[s] = input_vals[i]
    
print(cue_dict)

{'A': 17, 'B': 10, 'C': 4, 'D': 6, 'E': 15, 'F': 2, 'G': 7}


In [6]:
random.sample(all_syms, 3)

['C', 'E', 'A']

In [ ]:
itertools.product(all_syms, repeat=3)

In [17]:
len(list(itertools.product(all_syms, repeat=3)))

343

In [ ]:
others = []
for s in all_syms:
    t1 = generate_sequences(ops, random., len_seq = 2, cue_dict = cue_dict, init_values = s)
    
    

In [18]:
random.choice(all_syms)

'C'

In [13]:
# self
ops = ['+']
syms = all_syms
self_seqs = []
for sym in syms:
    self_trial = generate_sequences(ops, sym, len_seq = 2, cue_dict = cue_dict, init_values = sym)
    self_seqs.append(self_trial)

In [62]:
# other
init = ['A']
pad_syms = ['B', 'C']
other_seqs = []
for pad in pad_syms:
    other_trial = generate_sequences(ops, pad, len_seq = 2, cue_dict = cue_dict, init_values = sym)
    other_seqs.append(other_trial)
other_seqs

[[['A', ('+', 'B'), ('+', 'B'), 37]], [['A', ('+', 'C'), ('+', 'C'), 25]]]

In [ ]:
def generate_other(operators, inputs, init_values, rep = 1):
    # remove init form inputs
    seq = []
    combi_operators = list(itertools.product(operators, repeat=2))
    for _ in range(rep):
        for init in init_values:
            input_ids = [val for val in inputs if val != init]
            combi_inputcue = list(itertools.combinations(input_ids, r=2))
            cue = random.choice(combi_inputcue)
            op = random.choice(combi_operators)
            trial = [init,*zip(tuple(op), cue)]
            trial.append(calculate_output(trial, cue_dict, False))
            seq.append(trial)

    return seq

In [78]:
def generate_other_1step(operators, inputs, init_values):
    # remove init form inputs
    
    seq = []
    for init in init_values:
        input_ids = [val for val in inputs if val != init]
        cue = random.choice(input_ids)
        op = random.choice(operators)
        trial = [init,(op, cue)]
        trial.append(calculate_output(trial, cue_dict, False))
        seq.append(trial)
        cue = random.choice(input_ids)
        op = random.choice(operators)
        trial = [init,(op, cue)]
        trial.append(calculate_output(trial, cue_dict, False))
        seq.append(trial)

    return seq


In [80]:
cue_dict

{'A': 17, 'B': 10, 'C': 4, 'D': 6, 'E': 15, 'F': 2, 'G': 7}

In [82]:
generate_other_1step(['+'], ['A','B','C','D'], ['A','B','C','D','E'])

[['A', ('+', 'B'), 27],
 ['A', ('+', 'C'), 21],
 ['B', ('+', 'A'), 27],
 ['B', ('+', 'D'), 16],
 ['C', ('+', 'D'), 10],
 ['C', ('+', 'B'), 14],
 ['D', ('+', 'C'), 10],
 ['D', ('+', 'B'), 16],
 ['E', ('+', 'B'), 25],
 ['E', ('+', 'B'), 25]]

In [64]:
def generate_other(operators, inputs, init_values):
    # remove init form inputs
    
    seq = []
    combi_operators = list(itertools.product(operators, repeat=2))
    for init in init_values:
        input_ids = [val for val in inputs if val != init]
        combi_inputcue = list(itertools.combinations(input_ids, r=2))
        cue = random.choice(combi_inputcue)
        op = random.choice(combi_operators)
        seq.append([init,*zip(tuple(op), cue)])

    return seq


In [65]:
generate_other(['+'], ['A', 'B', 'C'], ['A', 'B', 'C'])

[['A', ('+', 'B'), ('+', 'C')],
 ['B', ('+', 'A'), ('+', 'C')],
 ['C', ('+', 'A'), ('+', 'B')]]

In [49]:
input_ids = ['A', 'B', 'C']
list(itertools.combinations(input_ids, 2))

[('A', 'B'), ('A', 'C'), ('B', 'C')]

In [66]:
def generate_self(operators, inputs, rep = 1):
    seq = []
    for _ in range(rep):
        for inp in inputs:
            trial = generate_sequences(operators, inp, len_seq = 2, cue_dict = cue_dict, init_values = inp)
            seq.append(trial)

    return seq


In [ ]:
generate_sequences(ops, sym, len_seq = 2, cue_dict = cue_dict, init_values = sym)

In [68]:
def generate_dummy(operators, inputs, init_values, rep = 1):
    # remove init form inputs
    
    seq = []
    combi_operators = list(itertools.product(operators, repeat=2))
    for _ in range(rep):
        for init in init_values:
            input_ids = [val for val in inputs if val != init]
            inp = random.choice(input_ids)
            cue = (inp, inp)
            op = random.choice(combi_operators)
            trial = [init,*zip(tuple(op), cue)]
            trial.append(calculate_output(trial, cue_dict, False))
            seq.append(trial)

    return seq

In [71]:
generate_dummy(ops, ['B', 'C', 'D', 'E', 'F'], ['B', 'C', 'D', 'E', 'F'])

[['B', ('+', 'F'), ('+', 'F'), 14],
 ['C', ('+', 'E'), ('+', 'E'), 34],
 ['D', ('+', 'B'), ('+', 'B'), 26],
 ['E', ('+', 'F'), ('+', 'F'), 19],
 ['F', ('+', 'E'), ('+', 'E'), 32]]